<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Name" data-toc-modified-id="Name-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Name</a></span></li><li><span><a href="#Search" data-toc-modified-id="Search-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Search</a></span><ul class="toc-item"><li><span><a href="#Load-Pre-cached-Results" data-toc-modified-id="Load-Pre-cached-Results-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Load Pre-cached Results</a></span></li><li><span><a href="#Run-From-Scratch" data-toc-modified-id="Run-From-Scratch-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Run From Scratch</a></span></li></ul></li><li><span><a href="#Analysis" data-toc-modified-id="Analysis-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Analysis</a></span><ul class="toc-item"><li><span><a href="#Gender-Breakdown" data-toc-modified-id="Gender-Breakdown-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Gender Breakdown</a></span></li><li><span><a href="#Face-Sizes" data-toc-modified-id="Face-Sizes-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Face Sizes</a></span></li><li><span><a href="#Appearances-on-a-Single-Show" data-toc-modified-id="Appearances-on-a-Single-Show-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Appearances on a Single Show</a></span></li><li><span><a href="#Screen-Time-Across-All-Shows" data-toc-modified-id="Screen-Time-Across-All-Shows-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Screen Time Across All Shows</a></span></li></ul></li><li><span><a href="#Persist-to-Cloud" data-toc-modified-id="Persist-to-Cloud-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Persist to Cloud</a></span><ul class="toc-item"><li><span><a href="#Save-Model-to-GCS" data-toc-modified-id="Save-Model-to-GCS-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Save Model to GCS</a></span><ul class="toc-item"><li><span><a href="#Make-sure-the-GCS-file-is-valid" data-toc-modified-id="Make-sure-the-GCS-file-is-valid-4.1.1"><span class="toc-item-num">4.1.1&nbsp;&nbsp;</span>Make sure the GCS file is valid</a></span></li></ul></li><li><span><a href="#Save-Labels-to-DB" data-toc-modified-id="Save-Labels-to-DB-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Save Labels to DB</a></span><ul class="toc-item"><li><span><a href="#Commit-the-person-and-labeler" data-toc-modified-id="Commit-the-person-and-labeler-4.2.1"><span class="toc-item-num">4.2.1&nbsp;&nbsp;</span>Commit the person and labeler</a></span></li><li><span><a href="#Commit-the-FaceIdentity-labels" data-toc-modified-id="Commit-the-FaceIdentity-labels-4.2.2"><span class="toc-item-num">4.2.2&nbsp;&nbsp;</span>Commit the FaceIdentity labels</a></span></li></ul></li></ul></li></ul></div>

In [ ]:
from esper.prelude import *
from esper.identity import *
from esper import embed_google_images

# Name

In [ ]:
name = 'Anderson Cooper'

# Search

## Load Pre-cached Results

In [ ]:
assert name != ''
results = FaceIdentityModel.load(name=name)
imshow(np.hstack([cv2.resize(x[1][0], (200, 200)) for x in results.model_params['images']]))
plt.show()
plot_precision_and_cdf(results)

## Run From Scratch

In [ ]:
assert name != ''
img_dir = embed_google_images.fetch_images(name)
face_imgs = load_and_select_faces_from_images(img_dir)
face_embs = embed_google_images.embed_images(face_imgs)
assert(len(face_embs) == len(face_imgs))
imshow(np.hstack([cv2.resize(x[0], (200, 200)) for x in face_imgs if x]))
plt.show()

In [ ]:
face_ids_by_bucket, face_ids_to_score = face_search_by_embeddings(face_embs)

In [ ]:
precision_model = PrecisionModel(face_ids_by_bucket)

In [ ]:
print('Select all MISTAKES. Ordered by DESCENDING score. Expecting {} frames'.format(precision_model.get_lower_count()))
lower_widget = precision_model.get_lower_widget()
lower_widget

In [ ]:
print('Select all NON-MISTAKES. Ordered by ASCENDING distance. Expecting {} frames'.format(precision_model.get_upper_count()))
upper_widget = precision_model.get_upper_widget()
upper_widget

In [ ]:
lower_precision = precision_model.compute_precision_for_lower_buckets(lower_widget.selected)
upper_precision = precision_model.compute_precision_for_upper_buckets(upper_widget.selected)

In [ ]:
precision_by_bucket = {**lower_precision, **upper_precision}

results = FaceIdentityModel(
    name=name, 
    face_ids_by_bucket=face_ids_by_bucket, 
    face_ids_to_score=face_ids_to_score,
    precision_by_bucket=precision_by_bucket, 
    model_params={
        'images': list(zip(face_embs, face_imgs))
    }
)
plot_precision_and_cdf(results)

In [ ]:
# Save the model
results.save()

# Analysis

## Gender Breakdown

In [ ]:
gender_breakdown = compute_gender_breakdown(results)

print('Raw counts:')
for k, v in gender_breakdown.items():
    print(' ', k, ':', v)
print()

print('Proportions:')
denominator = sum(v for v in gender_breakdown.values())
for k, v in gender_breakdown.items():
    print(' ', k, ':', v / denominator)
print()

In [ ]:
print('Showing examples:')
show_gender_examples(results)

## Face Sizes

In [ ]:
plot_histogram_of_face_sizes(results)

## Appearances on a Single Show

In [ ]:
show_name = 'Anderson Cooper 360'

In [ ]:
screen_time_by_video_id = compute_screen_time_by_video(results, show_name)

In [ ]:
plot_histogram_of_screen_times_by_video(name, show_name, screen_time_by_video_id)
plot_screentime_over_time(name, show_name, screen_time_by_video_id)

In [ ]:
plot_distribution_of_appearance_times_by_video(results, show_name)

## Screen Time Across All Shows

In [ ]:
screen_time_by_show = get_screen_time_by_show(results)

In [ ]:
plot_screen_time_by_show(name, screen_time_by_show)

# Persist to Cloud

## Save Model to GCS

In [ ]:
gcs_model_path = results.save_to_gcs()

### Make sure the GCS file is valid

In [ ]:
gcs_results = FaceIdentityModel.load_from_gcs(name=name)
plot_precision_and_cdf(gcs_results)

## Save Labels to DB

In [ ]:
from django.core.exceptions import ObjectDoesNotExist

def standardize_name(name):
    return name.lower()

person_type = ThingType.objects.get(name='person')

try:
    person = Thing.objects.get(name=standardize_name(name), type=person_type)
    print('Found person:', person.name)
except ObjectDoesNotExist:
    person = Thing(name=standardize_name(name), type=person_type)
    print('Creating person:', person.name)

labeler = Labeler(name='face-identity-{}'.format(person.name), data_path=gcs_model_path)

### Commit the person and labeler

In [ ]:
person.save()
labeler.save()

### Commit the FaceIdentity labels

In [ ]:
commit_face_identities_to_db(results, person, labeler)

In [ ]:
print('Committed {} labels to the db'.format(FaceIdentity.objects.filter(labeler=labeler).count()))